In [7]:
from typing import List, Dict, Any
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import TavilySearchResults
import os

from dotenv import load_dotenv
from langchain.agents import tool

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [51]:
@tool
def add(a: float, b: float):
    """Add two numbers."""
    return a + b

@tool
def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b

@tool
def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b


# 계산기 툴 함수
math_agent = create_react_agent(
    model="gpt-4o-mini",
    tools=[add, multiply, divide],
    prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks.\n"
        "- The user's question is given as input.\n"
        "- You MUST extract the exact numbers and operation from the question and perform ONLY that calculation.\n"
        "- DO NOT invent, generate, or solve any other math problems or examples.\n"
        "- If the question is ambiguous, ask for clarification.\n"
        "- After you're done with your task, respond to the supervisor directly.\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text.\n"
        "- 계산 결과를 반드시 메시지로 남겨"
    ),
    name="math_agent",
)

web_search = TavilySearchResults(k=3)

research_agent = create_react_agent(
    model="gpt-4o-mini",
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks. DO NOT do any math.\n"
        "- The user's question is given as input.\n"
        "- You MUST use the web_search tool to answer ONLY the user's question.\n"
        "- DO NOT invent, generate, or search for any unrelated information.\n"
        "- After you're done with your task, respond to the supervisor directly.\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)


In [19]:
from langchain.agents import AgentExecutor, ReActChain
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
tools = [add, multiply, divide]
prompt = """
You are a math agent.

INSTRUCTIONS:
- Assist ONLY with math-related tasks.
- The user's question is given as input.
- You MUST extract the exact numbers and operation from the question and perform ONLY that calculation.
- DO NOT invent, generate, or solve any other math problems or examples.
- If the question is ambiguous, ask for clarification.
- After you're done with your task, respond to the supervisor directly.
- Respond ONLY with the results of your work as a message.
"""

# 1) create_react_agent로 에이전트 생성
agent = create_react_agent(
    llm, 
    tools, 
    prompt=prompt
)



In [22]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [37]:
for chunk in agent.stream(
        {
            "messages": [
                {
                    "role": "user",
                    "content": "한개의 수식 (332 곱하기 7234 의 결과를 나누기 332) 순서대로 계산했을 때 얼마야?",
                }
            ]
        },
    ):
        pretty_print_messages(chunk, last_message=True)

final_message_history = chunk["agent"]["messages"]
print(final_message_history)

Update from node agent:


================================== Ai Message ==================================
Tool Calls:
  multiply (call_KLVQyiW6qxLjyA2tf5d3CIgN)
 Call ID: call_KLVQyiW6qxLjyA2tf5d3CIgN
  Args:
    a: 332
    b: 7234
  divide (call_P8Wu3k8byFYt02V0BYJOpFjt)
 Call ID: call_P8Wu3k8byFYt02V0BYJOpFjt
  Args:
    a: 332
    b: 332


Update from node tools:


================================= Tool Message =================================
Name: divide

1.0


Update from node agent:


================================== Ai Message ==================================
Tool Calls:
  divide (call_PRyL3kEyJQeCuzykGAu0wySj)
 Call ID: call_PRyL3kEyJQeCuzykGAu0wySj
  Args:
    a: 2401688
    b: 332


Update from node tools:


================================= Tool Message =================================
Name: divide

7234.0


Update from node agent:


================================== Ai Message ==================================

7234.0


[AIMessage(content='7234.0', additional_kwar

In [45]:
resutl =  agent.invoke(
        {
            "messages": [
                {
                    "role": "user",
                    "content": "한개의 수식 (332 * 7234 / 332) 순서대로 계산했을 때 얼마야?",
                }
            ]
        },
    )

final_message_history1 = chunk["agent"]["messages"]
print(final_message_history1)

[AIMessage(content='7234.0', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 319, 'total_tokens': 324, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bghox5cf7sXlvuuQ7kxhQFaDOq4G7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5652bfd9-e885-4825-aec6-06d458305e5c-0', usage_metadata={'input_tokens': 319, 'output_tokens': 5, 'total_tokens': 324, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]


In [52]:
import os
from langgraph_supervisor import create_supervisor
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

# OpenAI API 키 환경변수로 설정 필요
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

supervisor = create_supervisor(
    agents=[math_agent, research_agent],
    model=init_chat_model("gpt-4o-mini"),
    prompt=(
        "너는 두 명의 에이전트를 관리하는 슈퍼바이저야:\n"
        "사용자의 질문을 분석하여 필요한 도구들을 순차적으로 사용해 정보를 얻고, 종합적인 답변을 생성하세요.\n"
        "사용자 입력: {input}\n"
        "- research agent: 정보 탐색 관련 질문만 할당\n"
        "- math agent: 수학 계산 관련 질문만 할당\n"
        "아래 input은 사용자의 질문이다.\n"
        "input의 질문을 반드시 읽고, 적절한 에이전트에게 바로 할당해서 답변을 받아 사용자에게 전달하라.\n"
        "수학 질문이면 반드시 질문에서 숫자와 연산을 정확히 추출해서 math agent에게 그 계산만 하도록 지시해야 한다.\n"
        "질문과 무관한 임의의 수식이나 추가 계산을 생성하거나 할당하지 마라.\n"
        "계산 요청이 들어오면, 오직 그 계산만 수행해야 한다.\n"
        "직접 답변하지 말고 반드시 agent의 결과를 사용자에게 전달하라.\n"
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()




In [53]:

for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "오늘 날짜 알려줘, 3 곱하기 7은 얼마야?",
            }
        ]
    },
):
    pretty_print_messages(chunk, last_message=True)

final_message_history = chunk["supervisor"]["messages"]
print(final_message_history)

AttributeError: 'IsLastStepManager' object has no attribute 'update'